# Abgabe 1
Hinweise:
- Bearbeiten Sie alle Aufgaben bis zum **22.04.2025, 20 Uhr**.
- Bei den Aufgaben 1.2, und 3.2 und 4.2 soll ein (kleines) Gurobi-Programm geschrieben werden. Reichen sie dieses bitte als Jupyter Notebook ein. Die restlichen Abgaben können sie entweder als Markdown (Jupyter), pdf oder auch als Foto einer handgeschriebenen Lösung abgeben.
- Sie können die Lösung in Gruppen bearbeiten, aber jede/r sollte eine eigene Lösung abgeben.
- Als Teil der Klausurzulassung müssen sie 50\% der Aufgaben korrekt lösen. Versuchen sie aber, bei *allen* Aufgaben zumindest Lösungsansätze zu entwickeln.


# Aufgabe 1: Münzen
Folgendes Problem begegnet uns gelegentlich im täglichen Leben: Ein gegebener Geldbetrag, sagen wir 5.63€, soll aus möglichst wenigen Münzen gebildet werden. Es stehen dafür die üblichen Euromünzen zur Verfügung: 1 Cent, 2 Cent, 5 Cent, 10 Cent, 20 Cent, 50 Cent, 1 Euro, 2 Euro. 

1. Formulieren Sie das Problem als lineares Optimierungsproblem (Problemdaten/Indexmengen, Entscheidungsvariablen, Zielfunktion, Constraints).
2. Implementieren und lösen Sie das Problem mit Gurobi.


# Aufgabe 2: Modellierungstricks
1. Geben sie eine aus möglichst wenigen Münzen gebildet werden. Es stehen die , die zu den folgenden Nebenbedingung äquivalent sind $$ |x_1-x_2|\geq 5 \\ -3\leq x_1 \leq 3\\ -5\leq x_2 \leq 4 $$ äquivalent sind.

2. Linearisieren Sie den Term  
	$$y=x_1 \cdot x_2 \cdot x_3 $$
	wobei $x_1, x_2, x_3 \in \{0,1\}$. 


# Aufgabe 3: Standortproblem I
In einer Großstadt sollen öffentliche Ladesäulen für Elektroautos errichtet werden. Ein Stadtteil gilt dabei als versorgt, wenn sich die nächste Ladestation in weniger als 2km Entfernung befindet. Es gibt 30 Stadtteile und 50 mögliche Standorte für die Ladesäulen, deren Distanzen in der Datei `distances.csv` gegeben sind.

Wie können mit möglichst wenigen Ladesäulen sämtliche Stadtteile versorgt werden? 

1. Formulieren Sie das Problem als lineares Optimierungsproblem (Problemdaten/Indexmengen, Entscheidungsvariablen, Zielfunktion, Constraints)
2. Lösen Sie das Problem mit Gurobi und geben Sie die optimalen Standorte aus. 

# Aufgabe 4: Standortproblem II
Die Situation ist die gleiche wie in Aufgabe 4, nur dass die Stadtverwaltung festgestellt hat, dass sie nur Geld für 3 Ladesäulen besitzt. Dies reicht nicht zur Abdeckung aller Stadtteile. Wie sollten die 3 Ladesäulen platziert werden, so dass möglichst wenige Stadtteile unversorgt bleiben? 

1. Formulieren Sie das Problem als lineares Optimierungsproblem (Problemdaten/Indexmengen, Entscheidungsvariablen, Zielfunktion, Constraints)
2. Lösen Sie das Problem mit Gurobi. Geben Sie die optimalen Standorte sowie die unversorgten Stadtteile aus. 

___

## Aufgabe 1.1

# Das Münzproblem als Optimierungsproblem

Ein gegebener Geldbetrag soll aus möglichst wenigen Münzen gebildet werden. Es stehen dafür die üblichen Euromünzen zur Verfügung.

## Problemdaten:

* Der gegebene Geldbetrag, der gebildet werden soll. Wir nennen diesen Betrag $A$ (in Cents).
    * *Beispiel:* $A = 563$ Cents (für 5.63 €).
* Die Liste der verfügbaren Münzwerte. Wir nennen diese Werte $m_1, m_2, \dots, m_n$ (in Cents).
    * *Beispiel:* $m_1=1, m_2=2, m_3=5, m_4=10, m_5=20, m_6=50, m_7=100, m_8=200$. Hier ist $n=8$.

## Entscheidungsvariablen:

Wir müssen entscheiden, wie oft jede verfügbare Münzsorte verwendet werden soll.

* $x_i$: Anzahl der Münzen vom Wert $m_i$ für $i=1, \dots, n$.

## Zielfunktion:

Die Größe, die minimiert werden soll, ist die Gesamtzahl der verwendeten Münzen.

* Die Gesamtzahl der Münzen ist die Summe der Anzahlen jeder Sorte: $\sum_{i=1}^{n} x_i$.
* Die Zielfunktion lautet daher: Minimiere $\sum_{i=1}^{n} x_i$.

## Constraints:

Welchen Einschränkungen unterliegen die Anzahlen der Münzen ($x_i$)?

* Der Gesamtwert aller verwendeten Münzen muss exakt dem Zielbetrag $A$ entsprechen.
    $$ \sum_{i=1}^{n} m_i x_i = A $$
* Die Anzahl der Münzen jeder Sorte darf nicht negativ sein.
    $$ x_i \ge 0 \quad \text{für } i = 1, \dots, n $$

## Das gesamte Problem schreiben wir kompakt als:

$$
\min_{x_1, \dots, x_n} \sum_{i=1}^{n} x_i \\
\text{s.t.} \\
\sum_{i=1}^{n} m_i x_i = A \\
x_i \ge 0 \quad \text{für } i = 1, \dots, n \\
$$


## Aufgabe 1.2

In [11]:
import gurobipy as gp
from gurobipy import GRB

# --- Problemdaten ---
# Zielbetrag in Cents (z.B. 5.63 Euro)
ziel_betrag_cents = 563

# Verfügbare Münzwerte in Cents
muenz_werte = [1, 2, 5, 10, 20, 50, 100, 200]

# --- Gurobi Modell erstellen ---
model = gp.Model("CoinProblem")
model.setParam("OutputFlag", 0)  # Falls Gurobi Output erwünscht ist, diese Zeile auskommentieren

# --- Entscheidungsvariablen hinzufügen ---
# ein Dictionary, das Münzwert auf die Gurobi-Variable abbildet
# vtype=GRB.INTEGER stellt sicher, dass die Variablen ganzzahlig sind 
x = model.addVars(muenz_werte, vtype=GRB.INTEGER, name="x")

# --- Zielfunktion festlegen ---
# Minimieren der Gesamtzahl der Münzen: Summe aller Entscheidungsvariablen x[muenz_wert]
model.setObjective(x.sum(), GRB.MINIMIZE)

# --- Constraints hinzufügen ---
# Summe (Münzwert * Anzahl) = Zielbetrag
model.addConstr(
    gp.quicksum(muenz_wert * x[muenz_wert] for muenz_wert in muenz_werte) == ziel_betrag_cents,
    "TotalValueConstraint" 
)

# Optional:
# Die Nicht-Negativität (>= 0) ist durch vtype=GRB.INTEGER und den Standard lower bound von 0 abgedeckt
model.addConstrs((x[muenz_wert] >= 0 for muenz_wert in muenz_werte), "NonNegativity")


# --- Modell optimieren ---
print("Starte Optimierung...")
model.optimize()
print("Optimierung beendet.")
# --- Modell optimieren ---


print("\n" + "="*40) 
print(f"Ergebnis für den Betrag: {ziel_betrag_cents/100:.2f} Euro")
print("="*40) 

if model.Status == GRB.OPTIMAL:
    print("\nStatus: OPTIMAL")
    print("-"*40) 

    verwendete_muenzen_liste = []
    for muenz_wert in sorted(muenz_werte, reverse=True): # Durchläuft Münzarten, um Gurobi-Ergebnis abzufragen
        # Hier wird das OPTIMALE ERGEBNIS VON GUROBI abgerufen (.X)
        anzahl = round(x[muenz_wert].X)
        if anzahl > 0:
            verwendete_muenzen_liste.append((anzahl, muenz_wert))

    if verwendete_muenzen_liste:
        print("Details der Lösung:")
        # Max. Breite für Anzahlen und Werte ermitteln für saubere Ausrichtung
        # Diese Berechnung dient NUR der Formatierung der Ausgabe, NICHT der Lösungsfindung
        max_anzahl_len = max(len(str(anzahl)) for anzahl, wert in verwendete_muenzen_liste) if verwendete_muenzen_liste else 0
        max_wert_len = max(len(str(wert)) for anzahl, wert in verwendete_muenzen_liste) if verwendete_muenzen_liste else 0

        for anzahl, muenz_wert in verwendete_muenzen_liste:
            print(f"{anzahl:>{max_anzahl_len}} x {muenz_wert:<{max_wert_len}} Cent")

        print("-"*40) # Trennlinie
        # Die Gesamtzahl wird aus den von Gurobi gefundenen Anzahlen berechnet
        gesamtzahl_muenzen = sum(item[0] for item in verwendete_muenzen_liste)
        print(f"Gesamtzahl der Münzen: {int(gesamtzahl_muenzen)}") # Gesamtanzahl als Integer
    else:
        print("Keine Münzen benötigt (Zielbetrag war 0).")

# --- DEBUG ---
elif model.Status == GRB.INFESIBLE:
    print("\nStatus: UNLÖSBAR")
    print("-"*40)
    print(f"Der Betrag von {ziel_betrag_cents/100:.2f} Euro kann mit den gegebenen Münzen nicht exakt gebildet werden.")

elif model.Status == GRB.UNBOUNDED:
     print("\nStatus: UNBESCHRÄNKT") 
     print("-"*40)

else:
    print(f"\nStatus: Optimierung endete mit Statuscode {model.Status}")
    print("-"*40)
# --- DEBUG ---

print("="*40) 

Starte Optimierung...
Optimierung beendet.

Ergebnis für den Betrag: 5.63 Euro

Status: OPTIMAL
----------------------------------------
Details der Lösung:
2 x 200 Cent
1 x 100 Cent
1 x 50  Cent
1 x 10  Cent
1 x 2   Cent
1 x 1   Cent
----------------------------------------
Gesamtzahl der Münzen: 7


___

## Aufgabe 2.1

# Aufgabe 2: Modellierungstricks

## 2.1: Äquivalente lineare Nebenbedingungen

Gegeben sind die Nebenbedingungen:
$$
\begin{align*}
|x_1-x_2| &\geq 5 \\
-3 &\leq x_1 \leq 3 \\
-5 &\leq x_2 \leq 4
\end{align*}
$$

Die einfachen Schranken ($ -3 \leq x_1 \leq 3 $ und $ -5 \leq x_2 \leq 4 $) bilden bereits eine konvexe Menge. Die Herausforderung liegt in der Betragsungleichung $|x_1-x_2|\geq 5$.

Eine Ungleichung der Form $|a| \ge b$ (mit $b \ge 0$) ist äquivalent zu einer ODER-Bedingung:
$$ a \ge b \quad \text{ODER} \quad a \le -b $$
In unserem Fall:
$$ (x_1 - x_2 \ge 5) \quad \text{ODER} \quad (x_1 - x_2 \le -5) $$

mittels einer zusätzlichen **binären Variable** und der **Big-M-Methode** linearisieren da Menge nicht kponvex ist.

Wir führen eine binäre Hilfsvariable $ z \in \{0, 1\} $ ein.
* Wenn $ z=0 $, soll die Bedingung $ x_1 - x_2 \ge 5 $ gelten.
* Wenn $ z=1 $, soll die Bedingung $ x_1 - x_2 \le -5 $ gelten.

Mithilfe von Big-M-Konstanten ($M_1, M_2$), die groß genug gewählt werden, um die jeweils andere Bedingung zu "entspannen", erhalten wir die linearen Ungleichungen:
$$
\begin{align*}
x_1 - x_2 &\ge 5 - M_1 \cdot z \\
x_1 - x_2 &\le -5 + M_2 \cdot (1-z)
\end{align*}
$$

Die Werte für $M_1$ und $M_2$ können basierend auf den Schranken von $x_1$ und $x_2$ bestimmt werden, um die Ungleichungen gerade so weit zu entspannen, dass alle erlaubten Werte von $x_1, x_2$ die entspannte Ungleichung erfüllen:
* Maximale Differenz $x_1 - x_2$: $3 - (-5) = 8$.
* Minimale Differenz $x_1 - x_2$: $-3 - 4 = -7$.

Die Big-M-Werte ($M_1, M_2$) werden basierend auf den Schranken von $x_1$ ([-3, 3]) und $x_2$ ([-5, 4]) bestimmt:

* Für die Ungleichung $x_1 - x_2 \ge 5 - M_1 z$:
    Wenn $z=1$, entspannt sie zu $x_1 - x_2 \ge 5 - M_1$. Der minimale Wert von $x_1 - x_2$ innerhalb der Bounds ist $-7$. Um die Ungleichung zu entspannen, muss die rechte Seite kleiner oder gleich diesem Minimum sein: $5 - M_1 \le -7$. Dies ergibt $M_1 \ge 12$. Wir wählen $M_1 = 12$.

* Für die Ungleichung $x_1 - x_2 \le -5 + M_2 (1-z)$:
    Wenn $z=0$, entspannt sie zu $x_1 - x_2 \le -5 + M_2$. Der maximale Wert von $x_1 - x_2$ innerhalb der Bounds ist $8$. Um die Ungleichung zu entspannen, muss die rechte Seite größer oder gleich diesem Maximum sein: $-5 + M_2 \ge 8$. Dies ergibt $M_2 \ge 13$. Wir wählen $M_2 = 13$.

Die äquivalenten linearen Nebenbedingungen, die die ursprünglichen Bedingungen ersetzen, sind:

$$
\begin{align*}
-3 &\leq x_1 \leq 3 \\
-5 &\leq x_2 \leq 4 \\
x_1 - x_2 + 12 z &\ge 5 \\
x_1 - x_2 + 13 z &\le 8 \\ % Umgeformt aus x_1 - x_2 <= -5 + 13(1-z)
z &\in \{0, 1\}
\end{align*}
$$
Hier sind $x_1, x_2$ die ursprünglichen Variablen und $z$ ist eine neu eingeführte binäre Variable.

## 2.2: Linearisierung des Terms $y = x_1 \cdot x_2 \cdot x_3$ für $x_1, x_2, x_3 \in \{0,1\}$

Wir wollen den nicht-linearen Term $ y = x_1 \cdot x_2 \cdot x_3 $ linearisieren, wobei $ x_1, x_2, x_3 $ binäre Variablen sind ($0$ oder $1$). Das Ergebnis $y$ muss ebenfalls binär sein. Wir führen eine zusätzliche binäre Variable $y \in \{0,1\}$ ein, die das Produkt darstellen soll.

Die Linearisierung basiert auf folgenden logischen Bedingungen:
* $y$ soll $1$ sein, *genau dann wenn* $x_1=1$ UND $x_2=1$ UND $x_3=1$.
* Andernfalls ($mindestens$ einer der $x_i$ ist $0$), soll $y$ $0$ sein.

Wir erreichen dies durch folgende Menge linearer Nebenbedingungen:

$$
\begin{align*}
y &\le x_1 \\
y &\le x_2 \\
y &\le x_3 \\
y &\ge x_1 + x_2 + x_3 - 2 \\ % 2 = Anzahl der Variablen im Produkt - 1
y &\in \{0, 1\}
\end{align*}
$$

**Erläuterung der Nebenbedingungen:**

1.  $ y \le x_1, y \le x_2, y \le x_3 $: Diese drei Ungleichungen stellen sicher, dass $y$ gleich $0$ ist, wenn auch nur eine der Variablen $x_1, x_2, x_3$ gleich $0$ ist. Wenn z.B. $x_1=0$, dann muss $y \le 0$ gelten. Da $y$ eine binäre Variable ist (und somit nicht-negativ), erzwingt dies $y=0$.
2.  $ y \ge x_1 + x_2 + x_3 - 2 $: Diese Ungleichung stellt sicher, dass $y$ gleich $1$ ist, wenn alle $x_1, x_2, x_3$ gleich $1$ sind. Wenn $x_1=1, x_2=1, x_3=1$, dann $y \ge 1+1+1-2 = 1$. Da $y$ binär ist und nicht größer als 1 sein kann, muss $y=1$ sein. Wenn weniger als drei $x_i$ gleich $1$ sind (z.B. zwei sind 1, eins ist 0), dann ist die rechte Seite der Ungleichung $1+1+0-2=0$. $y \ge 0$ ist konsistent mit $y=0$, was der korrekte Wert für das Produkt ist.
3.  $ y \in \{0, 1\} $: Definiert $y$ als binäre Variable. (Die Variablen $x_1, x_2, x_3$ müssen ebenfalls als binär deklariert sein).


___

## Aufgabe 3

## Aufgabe 3.1: Formulierung als lineares Optimierungsproblem (ILP)

**Problembeschreibung:** Minimiere die Anzahl der Ladesäulen an 50 möglichen Standorten, sodass alle 30 Stadtteile versorgt sind (nächste Station $\le 2$ km entfernt).

### Problemdaten/Indexmengen

* Menge der Stadtteile (Nachfragepunkte): $ D = \{1, 2, \dots, 30\} $
* Menge der möglichen Standorte für Ladesäulen: $ L = \{1, 2, \dots, 50\} $
* Distanz zwischen Stadtteil $ i \in D $ und Standort $ j \in L $: $ d_{ij} $ (gegeben in `distances.csv`).
* Maximal zulässige Entfernung für Versorgung (Radius): $ R = 2 \, \text{km} $.

Basierend auf den Distanzdaten und dem Radius definieren wir für jeden Stadtteil $i$ die Menge der Standorte, die ihn versorgen können:
* $ L_i = \{j \in L \mid d_{ij} \le R\} \quad \text{für jedes } i \in D $

### Entscheidungsvariablen

Für jeden möglichen Standort $ j \in L $ müssen wir entscheiden, ob dort eine Ladesäule gebaut wird oder nicht.
* $ y_j = \begin{cases} 1 & \text{falls eine Ladesäule am Standort } j \text{ gebaut wird} \\ 0 & \text{sonst} \end{cases} \quad \text{für } j \in L $

Die Variablen sind binär: $ y_j \in \{0, 1\} $ für alle $ j \in L $.

### Zielfunktion

Wir wollen die Gesamtzahl der errichteten Ladesäulen minimieren.
* Minimiere $ \sum_{j \in L} y_j $

### Constraints (Nebenbedingungen)

Jeder Stadtteil $ i \in D $ muss versorgt werden. Ein Stadtteil $ i $ ist versorgt, wenn mindestens eine der Stationen, die ihn versorgen können (ein Standort $ j \in L_i $), gebaut wird ($y_j = 1$).
* Für jeden Stadtteil $ i \in D $: Die Summe der $ y_j $ für alle Standorte $ j $ in $ L_i $ muss mindestens $ 1 $ sein.
    $$ \sum_{j \in L_i} y_j \ge 1 \quad \text{für alle } i \in D $$


### Zusammenfassende mathematische Formulierung

$$
\begin{align*}
\min &\sum_{j \in L} y_j \\
\text{s.t.} \\
&\sum_{j \in L_i} y_j \ge 1 & \forall i \in D \\
\end{align*}
$$

 

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np 

# --- Problemdaten ---
distance_file = 'distances.csv' 
coverage_radius = 2.0 

# --- Daten lesen und verarbeiten ---
try:
    # pandas.read_csv nimmt standardmäßig die erste Spalte nicht als Index an, was hier passt.
    df_distances = pd.read_csv(distance_file)

    # --- IDs extrahieren ---
    try:
        standort_ids = [int(col.split()[-1]) for col in df_distances.columns]
        if len(standort_ids) != 50 or sorted(standort_ids) != list(range(1, 51)):
             print("Warnung: Die extrahierten Standort-IDs aus den Spaltennamen entsprechen nicht den erwarteten 1-50.")
             print(f"Extrahierte IDs: {standort_ids}")
    except Exception as e:
        print(f"Fehler beim Extrahieren der Standort-IDs aus den Spaltennamen: {e}")
        print("Stellen Sie sicher, dass die Spaltenüberschriften das Format 'Standort N' haben.")
        raise


    # Wenn 30 Zeilen nach dem Header da sind, sind es Stadtteile 1 bis 30.
    num_stadtteile = len(df_distances)
    if num_stadtteile != 30:
        print(f"Warnung: Erwartete 30 Stadtteile, aber {num_stadtteile} Zeilen in der Datei gefunden.")
        raise

    stadtteil_ids = list(range(1, num_stadtteile + 1))

    # --- Überdeckungs-Map erstellen ---
    # coverage_map: StadtteilID -> Liste der StandortIDs, die ihn versorgen können
    coverage_map = {st_id: [] for st_id in stadtteil_ids}

    # iterrows() ist für große DataFrames nicht am effizientesten, aber hier (30x50) in Ordnung
    for i, row in df_distances.iterrows(): 
        st_id = stadtteil_ids[i] 

        for j, so_id in enumerate(standort_ids):
             # Zugriff auf den Distanzwert in Zeile i, Spalte j
             dist_value = row.iloc[j] 
             try:
                 dist = float(dist_value)
                 if dist <= coverage_radius:
                     coverage_map[st_id].append(so_id)
             except (ValueError, TypeError):
                  print(f"Warnung: Ungültiger Distanzwert '{dist_value}' bei Stadtteil {st_id}, Standort {so_id}. Wird ignoriert.")
             except Exception as e:
                  print(f"Unerwarteter Fehler bei Distanzverarbeitung Stadtteil {st_id}, Standort {so_id}: {e}. Wert: '{dist_value}'")


except FileNotFoundError:
    print(f"Fehler: Datei '{distance_file}' nicht gefunden.")
    print("Bitte stellen Sie sicher, dass die Datei im selben Verzeichnis liegt oder geben Sie den vollständigen Pfad an.")
    exit()
except Exception as e: 
    print(f"Ein Fehler ist beim Lesen oder Verarbeiten der Distanzdaten aufgetreten: {e}")
    exit()

# --- Prüfung auf nicht versorgbare Stadtteile nach dem Parsen ---
uncoverable_districts = [st_id for st_id, covering_locs in coverage_map.items() if not covering_locs]
if uncoverable_districts:
    print(f"\nWARNUNG DES MODELLS: Folgende Stadtteile ({uncoverable_districts}) können von KEINEM Standort innerhalb von {coverage_radius}km versorgt werden.")
    print("Das bedeutet, dass das Problem UNLÖSBAR ist, da nicht alle Stadtteile versorgt werden können.")
    # Das Gurobi-Modell wird INFESIBLE, was wir im Ergebnis-Output abfangen.

# --- Gurobi Modell erstellen ---
model = gp.Model("LocationProblem")

# --- Parameter setzen (optional: Ausgabe unterdrücken) ---
model.setParam("OutputFlag", 0) 

# --- Entscheidungsvariablen hinzufügen ---
# y[j] = 1, wenn Standort j gewählt wird, 0 sonst
y = model.addVars(standort_ids, vtype=GRB.BINARY, name="y")

# --- Zielfunktion festlegen ---
# Minimiere die Gesamtzahl der gewählten Standorte
model.setObjective(y.sum(), GRB.MINIMIZE)

# --- Constraints hinzufügen ---
# Jeder Stadtteil muss versorgt werden
for st_id in stadtteil_ids:
    covering_locs = coverage_map[st_id]
    # Der Constraint: Summe der y_j für j in covering_locs[st_id] >= 1
    # Wenn covering_locs leer ist, wird die Summe 0, und 0 >= 1 macht das Problem unlösbar, was korrekt ist.
    model.addConstr(
        gp.quicksum(y[so_id] for so_id in covering_locs) >= 1,
        f"CoverDistrict_{st_id}" 
    )

# --- Modell optimieren ---
print("Starte Optimierung...")
model.optimize()
print("Optimierung beendet.")

# --- Ergebnisse ausgeben ---
print("\n" + "="*40)
print(f"Ergebnis des Standortproblems ({coverage_radius}km Radius)")
print("="*40)

if model.Status == GRB.OPTIMAL:
    print("\nStatus: OPTIMAL")
    print("-"*40)

    gewaehlte_standorte = [so_id for so_id in standort_ids if round(y[so_id].X) == 1]

    print(f"Minimale Anzahl benötigter Ladesäulen: {int(model.ObjVal)}")

    print("\nGewählte Standorte (ID):")
    if gewaehlte_standorte:
        gewaehlte_standorte_sortiert = sorted(gewaehlte_standorte)
        print(", ".join(map(str, gewaehlte_standorte_sortiert)))
    else:
        print("Keine Standorte benötigt (sollte nicht passieren bei >0 Stadtteilen, es sei denn, A=0).")

elif model.Status == GRB.INFESIBLE:
    print("\nStatus: UNLÖSBAR")
    print("-"*40)
    print(f"Es ist nicht möglich, alle Stadtteile im Umkreis von {coverage_radius}km zu versorgen.")
    if uncoverable_districts: # Gib die problematischen Stadtteile nur aus, wenn wir sie identifiziert haben
         print(f"Dies liegt wahrscheinlich daran, dass folgende Stadtteile nicht abgedeckt werden können: {uncoverable_districts}")


elif model.Status == GRB.UNBOUNDED:
     print("\nStatus: UNBESCHRÄNKT") 
     print("-"*40)

else:
    print(f"\nStatus: Optimierung endete mit Statuscode {model.Status}")
    print("-"*40)

print("="*40)

Starte Optimierung...
Optimierung beendet.

Ergebnis des Standortproblems (2.0km Radius)

Status: OPTIMAL
----------------------------------------
Minimale Anzahl benötigter Ladesäulen: 4

Gewählte Standorte (ID):
17, 18, 35, 49


___

## Aufgabe 4

## Aufgabe 4.1: Formulierung als lineares Optimierungsproblem

**Problembeschreibung:** Wähle genau 3 Standorte für Ladesäulen aus 50 möglichen Standorten, um die Anzahl der Stadtteile zu maximieren, die versorgt werden (nächste Station $\le 2$ km entfernt).

### Problemdaten/Indexmengen

* Menge der Stadtteile (Nachfragepunkte): $ D = \{1, 2, \dots, 30\} $
* Menge der möglichen Standorte für Ladesäulen: $ L = \{1, 2, \dots, 50\} $
* Distanz zwischen Stadtteil $ i \in D $ und Standort $ j \in L $: $ d_{ij} $ (gegeben in `distances.csv`).
* Maximal zulässige Entfernung für Versorgung (Radius): $ R = 2 \, \text{km} $.
* Maximale Anzahl an Ladesäulen (Budget): $ B = 3 $.

Basierend auf den Distanzdaten und dem Radius definieren wir wie zuvor für jeden Stadtteil $i$ die Menge der Standorte, die ihn versorgen können:
* $ L_i = \{j \in L \mid d_{ij} \le R\} \quad \text{für jedes } i \in D $

### Entscheidungsvariablen

Wir brauchen weiterhin Variablen, die entscheiden, wo gebaut wird, und zusätzliche Variablen, die den Erfolg der Abdeckung für jeden Stadtteil festhalten.

* $ y_j = \begin{cases} 1 & \text{falls eine Ladesäule am Standort } j \in L \text{ gebaut wird} \\ 0 & \text{sonst} \end{cases} \quad \text{für } j \in L $
* $ z_i = \begin{cases} 1 & \text{falls Stadtteil } i \in D \text{ versorgt wird} \\ 0 & \text{sonst} \end{cases} \quad \text{für } i \in D $

Alle Variablen sind binär: $ y_j \in \{0, 1\} $ für alle $ j \in L $ und $ z_i \in \{0, 1\} $ für alle $ i \in D $.

### Zielfunktion

Wir wollen die Anzahl der versorgten Stadtteile maximieren. Dies ist die Summe der $ z_i $-Variablen.

* Maximiere $ \sum_{i \in D} z_i $

### Constraints (Nebenbedingungen)

1.  **Budgetbeschränkung:** Es dürfen exakt $B=3$ Ladesäulen gebaut werden.
    $$ \sum_{j \in L} y_j = B $$
2.  **Abdeckungs-Verknüpfung:** Ein Stadtteil $i$ ($z_i=1$) kann nur als versorgt gelten, wenn mindestens eine Station, die ihn abdecken kann ($j \in L_i$), tatsächlich gebaut wird ($y_j=1$).
    * Die Summe der $ y_j $ für Standorte in $ L_i $ muss größer oder gleich $ z_i $ sein.
    $$ \sum_{j \in L_i} y_j \ge z_i \quad \text{für alle } i \in D $$
    *(Diese Bedingung stellt sicher: Wenn $\sum_{j \in L_i} y_j = 0$, dann $0 \ge z_i$, was $z_i=0$ erzwingt. Wenn $\sum_{j \in L_i} y_j \ge 1$, dann ist die Ungleichung erfüllt und der Optimierer kann $z_i=1$ wählen, um die Zielfunktion zu verbessern.)*

### Zusammenfassende mathematische Formulierung

$$
\begin{align*}
\max &\sum_{i \in D} z_i \\
\text{s.t.} \\
&\sum_{j \in L} y_j = B \\
&\sum_{j \in L_i} y_j \ge z_i & \forall i \in D \\
\end{align*}
$$


## Aufgabe 4.1


In [15]:
# Aufgabe 4.2

import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# --- Problemdaten ---
distance_file = 'distances.csv'
coverage_radius = 2.0 # km
max_stations = 3 # Budget

# --- Daten lesen und verarbeiten ---
try:
    df_distances = pd.read_csv(distance_file)

    # --- IDs extrahieren ---
    try:
        standort_ids = [int(col.split()[-1]) for col in df_distances.columns]
        if len(standort_ids) != 50 or sorted(standort_ids) != list(range(1, 51)):
             print("Warnung: Die extrahierten Standort-IDs aus den Spaltennamen entsprechen nicht den erwarteten 1-50.")
             print(f"Extrahierte IDs: {standort_ids}")
             # raise # Stoppen, wenn IDs nicht passen
    except Exception as e:
        print(f"Fehler beim Extrahieren der Standort-IDs aus den Spaltennamen: {e}")
        print("Stellen Sie sicher, dass die Spaltenüberschriften das Format 'Standort N' haben.")
        exit()

    num_stadtteile = len(df_distances)
    if num_stadtteile != 30:
        print(f"Warnung: Erwartete 30 Stadtteile, aber {num_stadtteile} Zeilen in der Datei gefunden.")
        # raise # Stoppen, wenn Anzahl Stadtteile nicht passt

    stadtteil_ids = list(range(1, num_stadtteile + 1))

    # --- Überdeckungs-Map erstellen ---
    # coverage_map: StadtteilID -> Liste der StandortIDs, die ihn versorgen können
    coverage_map = {st_id: [] for st_id in stadtteil_ids}

    for i, row in df_distances.iterrows():
        st_id = stadtteil_ids[i]

        for j, so_id in enumerate(standort_ids):
             dist_value = row.iloc[j]
             try:
                 dist = float(dist_value)
                 if dist <= coverage_radius:
                     coverage_map[st_id].append(so_id)
             except (ValueError, TypeError):
                  print(f"Warnung: Ungültiger Distanzwert '{dist_value}' bei Stadtteil {st_id}, Standort {so_id}. Wird ignoriert.")     
             except Exception as e:
                  print(f"Unerwarteter Fehler bei Distanzverarbeitung Stadtteil {st_id}, Standort {so_id}: {e}. Wert: '{dist_value}'")


except FileNotFoundError:
    print(f"Fehler: Datei '{distance_file}' nicht gefunden.")
    print("Bitte stellen Sie sicher, dass die Datei im selben Verzeichnis liegt oder geben Sie den vollständigen Pfad an.")
    exit()
except Exception as e:
    print(f"Ein schwerwiegender Fehler ist beim Lesen oder Verarbeiten der Distanzdaten aufgetreten: {e}")
    exit()

# Optional: Identifiziere Stadtteile, die überhaupt nicht abgedeckt werden können, unabhängig vom Budget
# Das Problem ist lösbar, auch wenn es solche Stadtteile gibt, sie werden einfach nicht versorgt (z_i=0).
uncoverable_by_any = [st_id for st_id, covering_locs in coverage_map.items() if not covering_locs]
if uncoverable_by_any:
     print(f"\nHinweis: Folgende Stadtteile ({uncoverable_by_any}) können von KEINEM Standort innerhalb von {coverage_radius}km versorgt werden.")
     print("Sie werden daher im Ergebnis immer unversorgt bleiben.")


# --- Gurobi Modell erstellen ---
model = gp.Model("MaxCoverageLocationProblem")

# --- Parameter setzen (optional: Ausgabe unterdrücken) ---
model.setParam("OutputFlag", 0) # 0 = aus, 1 = an

# --- Entscheidungsvariablen hinzufügen ---
# y[j] = 1, wenn Standort j gewählt wird, 0 sonst
y = model.addVars(standort_ids, vtype=GRB.BINARY, name="y")
# z[i] = 1, wenn Stadtteil i versorgt wird, 0 sonst
z = model.addVars(stadtteil_ids, vtype=GRB.BINARY, name="z")


# --- Zielfunktion festlegen ---
# Maximiere die Gesamtzahl der versorgten Stadtteile
model.setObjective(z.sum(), GRB.MAXIMIZE)

# --- Constraints hinzufügen ---
# 1. Budgetbeschränkung: Exakt 'max_stations' Standorte wählen
model.addConstr(y.sum() == max_stations, "BudgetConstraint")

# 2. Abdeckungs-Verknüpfung: Stadtteil i wird versorgt (z[i]=1) NUR wenn mindestens eine deckende Station (j in L_i mit y[j]=1) gewählt wird
for st_id in stadtteil_ids:
    covering_locs = coverage_map[st_id]
    # gp.quicksum über eine leere Liste ist 0, was korrekt ist (0 >= z[st_id] erzwingt z[st_id]=0)
    model.addConstr(
        gp.quicksum(y[so_id] for so_id in covering_locs) >= z[st_id],
        f"CoverLink_{st_id}"
    )

# --- Modell optimieren ---
print("Starte Optimierung...")
model.optimize()
print("Optimierung beendet.")

# --- Ergebnisse ausgeben ---
print("\n" + "="*40)
print(f"Ergebnis des Standortproblems II ({coverage_radius}km Radius, {max_stations} Stationen)")
print("="*40)

if model.Status == GRB.OPTIMAL:
    print("\nStatus: OPTIMAL")
    print("-"*40)

    # Anzahl versorgter Stadtteile (vom Zielwert)
    num_covered_districts = int(model.ObjVal)
    num_total_districts = len(stadtteil_ids)
    num_uncovered_districts = num_total_districts - num_covered_districts

    print(f"Maximale Anzahl versorgter Stadtteile: {num_covered_districts} von {num_total_districts}")
    print(f"Anzahl unversorgter Stadtteile: {num_uncovered_districts}")

    # Gewählte Standorte
    gewaehlte_standorte = [so_id for so_id in standort_ids if round(y[so_id].X) == 1]
    print(f"\nGewählte Standorte (ID):")
    if gewaehlte_standorte:
        print(", ".join(map(str, sorted(gewaehlte_standorte))))
    else:
         print("Keine Standorte gewählt (sollte nicht passieren bei Budget > 0).") # Sicherheit

    # Unversorgte Stadtteile
    unversorgte_stadtteile = [st_id for st_id in stadtteil_ids if round(z[st_id].X) == 0]
    print("\nUnversorgte Stadtteile (ID):")
    if unversorgte_stadtteile:
         print(", ".join(map(str, sorted(unversorgte_stadtteile))))
         # Überprüfen, ob die von KEINEM Standort abdeckbaren Stadtteile unter den unversorgten sind
         if uncoverable_by_any:
             always_uncovered_subset = all(item in unversorgte_stadtteile for item in uncoverable_by_any)
             if always_uncovered_subset:
                 print("(Beinhaltet Stadtteile, die von vornherein nicht versorgbar waren)")
             else:
                  print("(Einige Stadtteile, die prinzipiell versorgbar wären, bleiben unversorgt aufgrund des Budgetlimits)")

    else:
        print("Alle Stadtteile wurden versorgt (obwohl das Problemtext es anders vermuten ließ).") 

elif model.Status == GRB.INFESIBLE:
    print("\nStatus: UNLÖSBAR") 
    print("-"*40)
    print(f"Das Problem ist unlösbar. Dies sollte unter normalen Umständen nicht passieren.")


elif model.Status == GRB.UNBOUNDED:
     print("\nStatus: UNBESCHRÄNKT") 
     print("-"*40)

else:
    print(f"\nStatus: Optimierung endete mit Statuscode {model.Status}")
    print("-"*40)

print("="*40)

Starte Optimierung...
Optimierung beendet.

Ergebnis des Standortproblems II (2.0km Radius, 3 Stationen)

Status: OPTIMAL
----------------------------------------
Maximale Anzahl versorgter Stadtteile: 26 von 30
Anzahl unversorgter Stadtteile: 4

Gewählte Standorte (ID):
18, 21, 49

Unversorgte Stadtteile (ID):
5, 7, 16, 20
